<a href="https://colab.research.google.com/github/supplient/bachelor_design/blob/no_equal/Workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


# Clone repo

In [2]:
!git clone https://github.com/supplient/bachelor_design.git
import os
os.chdir("bachelor_design")
!pwd

Cloning into 'bachelor_design'...
remote: Enumerating objects: 153, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 396 (delta 90), reused 93 (delta 47), pack-reused 243
Receiving objects: 100% (396/396), 6.02 MiB | 33.30 MiB/s, done.
Resolving deltas: 100% (221/221), done.
/content/bachelor_design


In [3]:
!git checkout no_equal

Branch 'no_equal' set up to track remote branch 'no_equal' from 'origin'.
Switched to a new branch 'no_equal'


In [4]:
!git pull

Already up to date.


# Amount GDrive

In [5]:
from driver_amount import addh

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
[Colab] Using address head: /gdrive/My Drive


# Install python package

In [6]:
!pip install keras_bert

  Created wheel for keras-bert: filename=keras_bert-0.84.0-cp36-none-any.whl size=36139 sha256=c6ea75c84581726fa781492f3fcae4966a4665e97d436663592b40de9c8115ed
  Stored in directory: /root/.cache/pip/wheels/1f/59/04/12e95257aebfd27f7edaaf65ab7dd57b5f6cadfb183f1a4ccd
  Created wheel for keras-transformer: filename=keras_transformer-0.37.0-cp36-none-any.whl size=12942 sha256=ea2dc3f86b369a99d3f57b2b066a7982da863818335d9975be7f9bb633a72d3d
  Stored in directory: /root/.cache/pip/wheels/2a/f9/31/2a3289e948852ce0dd3fcd94c34bbc7eb9628842cb7110a87b
  Created wheel for keras-pos-embd: filename=keras_pos_embd-0.11.0-cp36-none-any.whl size=7554 sha256=6a70e7ed620d9cd42000227e6231ec1a867d4790dd0fc3fe2688f0fda175e9b3
  Stored in directory: /root/.cache/pip/wheels/5b/a1/a0/ce6b1d49ba1a9a76f592e70cf297b05c96bc9f418146761032
  Created wheel for keras-multi-head: filename=keras_multi_head-0.27.0-cp36-none-any.whl size=15611 sha256=5869a2c6a76ab824f628e4705ba3a299a546413bff8619a461f2f5f1af660237
  Stor

In [7]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-_a_9hl17
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-_a_9hl17
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=f57c7a6c8aaa31892c28605d81705bb61a348944d6e7549ed52c525a1bd9e468
  Stored in directory: /tmp/pip-ephem-wheel-cache-r8fy7u4c/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [0]:
import os
os.chdir("/usr/local/lib/python3.6/dist-packages/keras_contrib/layers/")
lines = []
with open("crf.py", "r") as fd:
  lines = fd.readlines()
lines[515] = "            mask2 = K.cast(K.concatenate([mask, K.cast(K.zeros_like(mask[:, :1]), mask.dtype)], axis=1),\n"
with open("crf.py", "w") as fd:
  fd.writelines(lines)

os.chdir("/content")

In [0]:
os.chdir("bachelor_design")

# Test Data

In [0]:
input_strs = [
"通过SNMP协议从设备直接采集告警",
"通过Socket接口从设备直接采集告警",
"通过Restful服务接口采集告警",
"通过API接口采集告警",
"通过网关北向接口采集告警",
"通过telnet远程登录执行命令并分析返回结果采集告警",
"通过发送并分析sip信令采集告警",
"根据大数据指标趋势进行智能预警",
"根据用户投诉人工录入告警信息",
"重点号码监控生成的告警信息接入",
"从各种接口输入的告警数据适配接口协议解析告警信息",
"标准告警字段及字典定义",
"告警级别的重新定义",
"通过把设备标注为工程状态不对此设备产生告警信息",
"对于已经产生的告警信息可以手工标注为工程告警",
"核心网告警格式到标准告警格式的映射规则定义",
"接入网告警格式到标准告警格式的映射规则定义",
"承载网告警格式到标准告警格式的映射规则定义",
"核心网告警根据映射规则转换为标准告警",
"接入网告警根据映射规则转换为标准告警",
"承载网告警根据映射规则转换为标准告警",
"原始告警转化为标准告警",
"定义告警主从关联规则",
"根据主从规则将符合规则条件的告警建立起主从关系",
"定义由阈值触发的衍生关联规则阈值参数设置",
"根据阈值触发的衍生规则将符合规则条件的告警衍生新告警并建立关联关系",
"定义由同源触发的衍生关联规则阈值参数设置",
"根据阈值触发的衍生规则将符合规则条件的告警衍生新告警并建立关联关系",
"网元内部的告警关联关系的建立",
"网元间告警关联关系的建立",
"关联关系参数设置",
"将设备的告警转换成影响业务的告警",
"以树形的方式按层次呈现告警信息",
"在网络拓扑图上呈现告警信息",
"在GIS地图上呈现告警信息",
"在图表上呈现告警信息",
"告警流水窗提供对告警属性快速筛选功能并支持多列组合进行筛选多列同时筛选时条件是取与的关系提供自定义筛选过滤的方式可以对一列添加多个筛选条件",
"将符合条件的告警通过短信推送到运维人员的手机上",
"以声音闪光的方式提示告警",
"系统支持设置监控视图各个流水窗口的最大显示告警数量",
"根据当班人员所关心的专业网元类型等设置窗口显示过滤器将符合设置条件的告警信息以及专业内的关联告警信息分别实时呈现在相应告警子窗口中",
"告警流水窗支持点击表头可以对该列进行正序和反序的排序功能告警流水窗提供对告警属性快速筛选功能并支持多列组合进行筛选多列同时筛选时条件是取与的关系提供自定义筛选过滤的方式可以对一列添加多个筛选条件",
"告警流水窗支持点击表头可以对该列进行正序和反序的排序功能告警流水窗提供对告警属性快速筛选功能并支持多列组合进行筛选多列同时筛选时条件是取与的关系提供自定义筛选过滤的方式可以对一列添加多个筛选条件",
"告警流水窗支持点击表头可以对该列进行正序和反序的排序功能告警流水窗提供对告警属性快速筛选功能并支持多列组合进行筛选多列同时筛选时条件是取与的关系提供自定义筛选过滤的方式可以对一列添加多个筛选条件",
"为了方便监控人员对选中的告警进行操作系统默认对选择的告警进行屏幕锁定可以用解除锁定按钮来解除屏幕锁定如果选择流水列表中的具体的告警支持多选则锁定所选择告警不影响告警流水列表的告警监控",
"告警自动确认规则的配置",
"满足规则的告警自动确认不需要人工确认",
"人工确认告警",
"批量进行告警的人工确认",
"配置自动清除告警的规则",
"满足自动清除规则的告警自动清除",
"人工清除告警",
"人工批量清除告警",
"根据派单规则设置告警发生后自动派单",
"不满足派单规则的告警人工进行派单操作",
"告警工单的流程化处理操作",
"按照告警类型统计告警生成报表",
"按照告警级别统计告警生成报表",
"按照专业统计告警生成报表",
"按照时间段统计告警生成报表",
"按照厂家统计告警生成报表",
"按照地域统计告警生成报表",
"按照网元统计告警生成报表",
"统计存在关联关系的告警生成报表",
"统计自动派单的告警生成报表",
"统计人工派单的告警生成报表",
"统计告警派单的处理结果情况生成报表",
"在省际拓扑视图上展现全业务监测",
"在省内拓扑视图上展现全业务监测",
"把网络路由情况展现在拓扑视图上",
"需要进行巡检的网元范围设定",
"巡检指标的定义出现次数定义",
"主要设置巡检的时间频次等内容",
"查看巡检之后的结果可按照时间段任务名称等进行查询",
"按照既定格式输出巡检报告",
"将巡检出的问题通过告警推送",
"厂家Corba接口私有OMC接口指令文件数据库由采集平台提供实时性能指标数据",
"性能监控只针对具有实时特性的指标进行监控主要进行24小时内时间粒度为1小时以下含的性能指标进行实时监控",
"集中性能监控的性能数据是要都要在一个周期内呈现例如800-815的性能数据要在830前呈现如果符合条件则要在830前发出性能告警",
"进行数据分析",
"性能数据的存储",
"自动计算基线和容忍度",
"手工设置性能指标的阈值区间以此区间范围计算性能指标的级别",
"多指标关联告警能力说明1告警标题可设置告警的标题名称2告警级别可根据不同条件范围设置不同的告警级别支持四个级别3告警指标对于相同类型同时采集的指标可选择告警条件指标并设置告警条件4网元条件能对告警条件适用的网元范围进行设置网元条件包括厂家地区网元名称不指定则表示对所有符合指标条件的网元进行告警判定5时间条件限定哪些时间点的性能数据进行性能告警判断不指定或全部选择则表示所有时间性能指标进行告警判定6告警正文设置正文中显示内容包括当前kpi指标值",
"实时性能告警发生后会根据规则自动清除告警清除规则有两种形式1性能指标恢复不满足告警阈值则系统发出性能告警清除消息清除之前发送的性能告警2告警清除观察期性能指标恢复不满足告警阈值并且在一定的时间范围内未再发生恶化则系统发出性能告警消息清除之前发送的性能告警",
"告警流水视图适用于大数量设备主要应用方式是对于已经产生性能预警的设备进行分析处理对于设备类型多不关注所有设备情况或只关注有告警劣化设备适用进行问题结果监控则采用告警流水方式监控",
"矩阵监控视图主要用于监控少量重点关注类型设备对于设备类型数量少或小网元挂在大网元下的情况适用按总体结合细节钻取监控可以通过矩阵一屏显示则采用矩阵方式监控",
"系统提供指标导航树用图表方式显示实时性能指标数据具体功能要求如下1趋势图以曲线方式显示并提供柱图显示切换；2趋势图能够缩放将比较密集集中的内容放大显示；3趋势图提供对比功能可以选择当前系统中存储的历史数据与已经显示的性能数据进行对比；4在矩阵告警详情性能详情告警流水中能够查看相关指标的趋势图；",
"定义多种开户模板简化开户填写内容",
"定义有附加属性的业务模板支持快速开通业务",
"向核心网和接入设备下发开户指令完成开户",
"下载开户模板导入批量开户数据进行开户操作",
"向核心网和接入设备下发用户销户指令",
"对已开户的部门号码进行批量销户操作",
"修改用户基础信息设置基本通话功能",
"更换接入设备",
"用户变更号码",
"开通修改关闭被叫一号通业务",
"配置核心网的业务包括各类权限通话功能代答来电显示秘书等业务",
"将业务工单转换成指令数据向核心网下发",
"接入设备的适配如TELNETSSHHTTP等方式实现接入设备的开局和增删用户配置",
"根据业务向接入设备进行下发",
"开通修改关闭呼叫转移业务",
"从其他系统资源将通讯录信息同步至服务器",
"根据排重规则进行通讯录数据排重",
"添加修改删除单个通讯录数据",
"按导入模板批量导入通讯录数据",
"所有的数据进行审核或驳回",
"提供多种IP话机的协议适配接口支持多种IP话机接入",
"根据通讯录客户端请求下发全量通讯录数据",
"根据通讯录客户端请求条件下发增量通讯录数据",
"接入通讯录服务器的安全认证处理",
"向通讯录服务器发送全量数据同步请求并存储到本地通讯录",
"向通讯录服务器发送增量数据同步请求并更新本地通讯录",
"根据关键字查询本地的通讯录信息",
"根据Android系统来电通知将来电号码关联出通讯录内的该号码详细信息",
"放号模板定义了SIP号码的各项配置参数包括注册密码用户密码呼叫权限等在创建号码时关联相应的放号模板可以快速添加号码",
"单个增加用户号码",
"按照起止码批量创建号码",
"系统支持批量导入号码按照提供的Excel模板录入号码信息可以快速创建",
"批量设置号码IPT权限",
"修改或者删除号码资源",
"批量删除号码资源",
"对各部门的号段进行分配管理",
"预留号码进行维护",
"特服号码的维护管理",
"需要重点保障的号码管理",
"部门对可用号码资源的查询禁用筛选靓号匹配",
"部门对已经使用的号码查询",
"各部门对可用号码可进行禁用授权使用操作",
"查询经销户后处于冻结期的号码资源",
"对号码资源的调拨处理部门之间的调拨",
"按照号码尾号号段靓号规则号码状态等条件进行查询统计",
"核心网设备资源模型的建立",
"承载网设备资源模型的建立",
"接入网设备资源模型的建立",
"终端设备资源模型的建立",
"根据资源模型填写参数创建设备",
"从外部批量导入设备",
"修改设备参数删除设备",
"设备之间的关联关系维护",
"设备的统计查询生成报表",
"定义用户对象的模型如基本信息设备信息职位信息关联关系等",
"查询开户销户的所有用户",
"随时查询某个用户的注册状态",
"查询使用的设备信息",
"网管资源模型的建立",
"各网管参数的维护",
"网管各个接口的维护",
"网管各个接口参数的维护",
]

# Get input

In [0]:
from cut_and_tag import load_stopwords, cut_and_remove_stopwords
import config

In [12]:
stopwords = load_stopwords(addh + config.STOPWORDS_PATH)
cut_seqs = []
char_seqs = []
for input_str in input_strs:
    cut_seq = cut_and_remove_stopwords(input_str, stopwords)
    cut_seqs.append(cut_seq)
    
    char_seq = []
    for w in cut_seq:
        for c in w:
            char_seq.append(c)
    char_seqs.append(char_seq)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.776 seconds.
Prefix dict has been built successfully.


# Load params

In [0]:
import json

In [0]:
tag_vocab = None
with open(addh + config.TAG_VOCAB_PATH, "r") as fd:
    tag_vocab = json.load(fd)

# Identify function points

## Load model

In [16]:
import keras
import keras_bert
from keras_contrib.layers import CRF

Using TensorFlow backend.


In [17]:
bert_model, bert_model_config = keras_bert.build_model_from_config(
    # config_file
    addh + config.BERT_CONFIG_PATH, 
    # settings
    training=False, # Not train the whole model. Ignore NSP and MLM
    trainable=True
)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
input_token = keras.layers.Input(shape=(config.SEQ_LEN,))
input_segment = keras.layers.Input(shape=(config.SEQ_LEN,))

bert_output = bert_model([input_token, input_segment])

crf_model = CRF(len(tag_vocab), sparse_target=True)

output = crf_model(bert_output)

model = keras.models.Model([input_token, input_segment], output)

In [0]:
model.load_weights(addh + config.MODEL_PATH)

## Preprocess

In [0]:
import preprocess

In [0]:
vocab = keras_bert.load_vocabulary(addh + config.BERT_VOCAB_PATH)

token_id_seqs = preprocess.preprocess_char(
    char_seqs,
    vocab,
    config.SEQ_LEN,
    True
)
segment_seqs = preprocess.create_segment(
    len(token_id_seqs),
    len(token_id_seqs[0])
)

## Predict

In [0]:
batch_size = 16
completed = False
import tensorflow as tf

In [23]:
output_one_hot_seqs = None
while not completed:
    try:
        output_one_hot_seqs = model.predict(
            [token_id_seqs, segment_seqs],
            batch_size = batch_size,
            verbose=1
        )
        completed = True
    except tf.errors.ResourceExhaustedError as e:
        batch_size = int(batch_size/2)
        print("Batch Size too large, turn to " + str(batch_size))
        if batch_size < 1:
            raise e


150/150 [==============================] - 9s 62ms/step


## Postprocess

In [0]:
from epoch_checkpoint import judgeWhichTag
import numpy as np

In [0]:
# Turn to tag id
output_seqs = []
for one_hot_seq in output_one_hot_seqs:
    one_hot_seq = one_hot_seq[1:-1] # Remove [CLS] and [SEP]
    seq = [[np.argmax(x)] for x in one_hot_seq]
    output_seqs.append(seq)

In [0]:
rev_tag_vocab = {}
for key, value in tag_vocab.items():
    rev_tag_vocab[value] = key

In [0]:
# Judge each output seq's tag
output_tags = [judgeWhichTag(seq, rev_tag_vocab) for seq in output_seqs]

In [0]:
# Count each tag category's number
tag_num = {}
for tag in output_tags:
    num = tag_num.get(tag, 0)
    tag_num[tag] = num + 1

In [0]:
fp_counts = tag_num

# Calculate cost

In [0]:
from funcomo import FUNCOMO

## Set model params

In [0]:
cost_model_params = {
    "fp_p" : FUNCOMO.fp_p,
    "fp_q" : FUNCOMO.fp_q,
    "VAF" : [3, 3, 3,
          3, 3, 3,
          3, 3, 3,
          3, 3, 3,
          3, 3,],

    "lang_factor" : FUNCOMO.lang_factor["C++"],

    "develop_mode_factor" : FUNCOMO.develop_mode_factor["semi-detached"],
    "EAF" : {
          "required software reliability": 1.15,
          "run-time performance constraints": 1.11,
          "analyst capaility": 0.86,
          "applications experience": 0.82,
          "required development schedule": 1.04,
    },
}

## Use cost model

In [0]:
cmp = cost_model_params

AFP = FUNCOMO.FP2AFP(
    fp_counts,
    cmp["fp_p"],
    cmp["fp_q"],
    cmp["VAF"],
)

KLOC = FUNCOMO.scale2KLOC(
    AFP,
    cmp["lang_factor"],
)

PM = FUNCOMO.KLOC2PM(
    KLOC,
    cmp["develop_mode_factor"][0],
    cmp["develop_mode_factor"][1],
    cmp["EAF"],
)

## Output result

In [0]:
from pprint import PrettyPrinter

In [0]:
pp = PrettyPrinter(indent = 4)

In [35]:
print("Using params: ")
pp.pprint(cost_model_params)

Using params: 
{   'EAF': {   'analyst capaility': 0.86,
               'applications experience': 0.82,
               'required development schedule': 1.04,
               'required software reliability': 1.15,
               'run-time performance constraints': 1.11},
    'VAF': [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
    'develop_mode_factor': (3.0, 1.12),
    'fp_p': 0.65,
    'fp_q': 0.01,
    'lang_factor': 0.064}


In [36]:
print("Result: ")
pp.pprint({
    "KLOC": KLOC,
    "PM": PM,
})

Result: 
{'KLOC': 10.272, 'PM': 38.154110117576785}
